In [1]:
import pymysql

In [ ]:
# Manage connection to SQL db

def connect_db():
	return pymysql.connect(
		host = 'localhost',
		user = 'root',
		password = '',
		database = 'navy',
		cursorclass=pymysql.cursors.DictCursor
	)

def close_db(connection):
	print("May the winds be faithful to your cause")
	connection.close()

In [3]:
# Helper functions for query management

import pymysql

# Function to execute a query
def execute_query(con:pymysql.Connection, query, values=None):
    try:
        with con.cursor() as cursor:
            if values:
                cursor.execute(query, values)
            else:
                cursor.execute(query)
            con.commit()
            print("Query executed successfully!")
    except pymysql.MySQLError as e:
        print(f"Error executing query: {e}")

# Function to insert data
def insert_data(con, table, data):
    columns = ', '.join(data.keys())
    placeholders = ', '.join(['%s'] * len(data))
    query = f"INSERT INTO {table} ({columns}) VALUES ({placeholders})"
    execute_query(con, query, tuple(data.values()))

# Function to delete data
def delete_data(con, table, condition_column, condition_value):
    query = f"DELETE FROM {table} WHERE {condition_column} = %s"
    execute_query(con, query, (condition_value,))

# Function to update data
def update_data(con, table, updates, condition_column, condition_value):
    set_clause = ', '.join([f"{col} = %s" for col in updates.keys()])
    query = f"UPDATE {table} SET {set_clause} WHERE {condition_column} = %s"
    values = list(updates.values()) + [condition_value]
    execute_query(con, query, values)
    
def get_last_inserted_id(con):
	query = "SELECT LAST_INSERT_ID()"
	with con.cursor() as cursor:
		cursor.execute(query)
		result = cursor.fetchone()
		return result[0]

In [ ]:
# add entities based on tables defined in database.py

def add_ship(connection):
  ship_details = input("Enter ship details (format: \"<name>, <type>, <year_of_build>, <origin_shipyard>, <gun_count>, <coordinates>, <last_port_of_call>, <last_date_of_port>\"):")
  insert_data(connection, "SHIP", ship_details)

def add_officer(connection):
  officer_details = input("Enter officer details (format: \"<name>, <age>, <rank_index>, <birthplace>, <status>\"):")
  insert_data(connection, "OFFICER", officer_details)


def make_flag_officer(connection):
	add_officer(connection)
	officer_id = str(get_last_inserted_id(connection))

	flag_details = ", ".join(officer_id, input("Enter flag-specific details (format: \"<title>, <squadron>, <flagship_id>, <predecessor_id>\"):"))

	insert_data(connection, "FLAG_OFFICER", flag_details)
	print("Flag officer added successfully!")
    
def make_commissioned_officer(connection):
	add_officer(connection)
	officer_id = str(get_last_inserted_id(connection))

	commissioned_details = ", ".join(officer_id, input("Enter commissioned-specific details (format: \"<seniority>, <ship_id>, <position>\"):"))

	insert_data(connection, "COMMISSIONED_OFFICER", commissioned_details)	
	print("Commissioned officer added successfully!")

def make_warrant_officer(connection):
	add_officer(connection)
	officer_id = str(get_last_inserted_id(connection))

	warrant_details = ", ".join(officer_id, input("Enter warrant-specific details (format: \"<ship_id>, <role>, <appointing_agency>\"):"))

	insert_data(connection, "WARRANT_OFFICER", warrant_details)
	print("Warrant officer added successfully!")

def make_petty_officer(connection):
	add_officer(connection)
	officer_id = str(get_last_inserted_id(connection))

	details = ", ".join(officer_id, input("Enter petty-specific details (format: \"<ship_id>, <rating>\"):"))

	insert_data(connection, "PETTY_OFFICER", details)
	print("Petty officer added successfully!")

def add_crew(connection):
  crew_details = input("Enter crew details (format: \"<name>, <age>, <role>, <status>\"):")
  insert_data(connection, "CREW", crew_details)

In [5]:
def add_dispatch(connection):
	dispatch_details = input("Enter dispatch details (format: \"<Date_Issued>, <Issuing_Officer>, <Orders>, <Dispatch_Vessel>\"):")
	insert_data(connection, "DISPATCH", dispatch_details)
	dispatch_id = get_last_inserted_id(connection)

	receiving_ships = input("Enter receiving ships (format: \"<ship_id1>, <ship_id2>, ...\"):")
	receiving_ships = receiving_ships.split(", ")
	
	for ship_id in receiving_ships:
	    insert_data(connection, "SHIPS_RECEIVED", {"Dispatch_ID": dispatch_id, "Ship_ID": ship_id})
	
	print("Dispatch added successfully!")

def add_enemy_ship(connection):
    enemy_ship_details = input("Enter enemy ship details (format: \"<Nationality>, <Threat_Level>, <Last_Reported_By>, <Last_Sighted_At>, <Last_Sighting>, <Current_Status>\"):")
    insert_data(connection, "ENEMY_SHIP", enemy_ship_details)
    print("Enemy ship added successfully!")

def add_squadron(connection):
	squadron_details = input("Enter squadron details (format: \"<Fleet>, <Commander>, <Station>, <S_Status>\"):")
	insert_data(connection, "SQUADRON", squadron_details)
	squadron_id = get_last_inserted_id(connection)
    
	ships = input("Enter ships in squadron (format: \"<ship_id1>, <ship_id2>, ...\"):")
	ships = ships.split(", ")
	for ship_id in ships:
	    insert_data(connection, "SHIPS_IN_SQUADRON", {"Squadron_ID": squadron_id, "Ship_ID": ship_id})

	print("Squadron added successfully!")

def add_fleet(connection):
    fleet_details = input("Enter fleet details (format: \"<Station>, <Commander_in_Chief>\"):")
    insert_data(connection, "FLEET", fleet_details)
    print("Fleet added successfully!")

def add_report(connection):
    report_details = input("Enter report details (format: \"<Dispatch_ID>, <Reporting_Officer>, <R_Status>, <Description>\"):")
    insert_data(connection, "REPORT", report_details)
    print("Report added successfully!")

def add_engagement(connection):
	engagement_details = input("Enter engagement details (format: \"<Coordinates>, <Time>, <Casualties>, <Outcome>\"):")
	insert_data(connection, "ENGAGEMENT", engagement_details)

	engagement_id = get_last_inserted_id(connection)

	ships = input("Enter British Royal Navy ships involved in the engagement (format: \"<ship_id1>, <ship_id2>, ...\"):")
	ships = ships.split(", ")
	for ship_id in ships:
		insert_data(connection, "SHIPS_ENGAGED", {"Engagement_ID": engagement_id, "Ship_ID": ship_id})

	ships = input("Enter enemy ships involved in the engagement (format: \"<enemy_ship_id1>, <enemy_ship_id2>, ...\"):")
	ships = ships.split(", ")
	for ship_id in ships:
		insert_data(connection, "ENEMY_SHIPS_ENGAGED", {"Engagement_ID": engagement_id, "Enemy_ID": ship_id})
	
	print("Engagement added successfully!")

def add_station(connection):
    station_details = input("Enter station details (format: \"<Name>, <Coordinates>\"):")
    insert_data(connection, "STATION", station_details)
    print("Station added successfully!")

def add_port(connection):
	port_details = input("Enter port details (format: \"<Name>, <Coordinates>, <Alignment>\"):")
	insert_data(connection, "PORT", port_details)
	port_id = get_last_inserted_id(connection)
	
	supplies = input("Enter supplies available at the port (format: \"<supply1>, <supply2>, ...\"):")
	supplies = supplies.split(", ")
    
	for supply in supplies:
		insert_data(connection, "SUPPLIES_AT_PORT", {"Port_ID": port_id, "Supply": supply})

	print("Port added successfully!")

In [ ]:
# Modification/Update queries

def promote_officer(connection):
	officer_id = input("Enter officer ID to promote:")
	rank = input("Enter new rank:")
	update_data(connection, "OFFICER", {"Rank": rank}, "Officer_ID", officer_id)
	print("Officer promoted successfully!")

def update_enemy_info(connection):
	enemy_id = input("Enter enemy ship ID to update:")

	

	enemy_ship_details = input("Enter enemy ship details (format: \"<Nationality>, <Threat_Level>, <Last_Reported_By>, <Last_Sighted_At>, <Last_Sighting>, <Current_Status>\"):")
	update_data(connection, "ENEMY_SHIP", enemy_ship_details, "Enemy_ID", enemy_id)
    print("Enemy ship added successfully!")

In [7]:
from database import create_all_tables

def main():

	print("Greetings, welcome to the Naval Archives.")
	connection = connect_db();

	create_all_tables(connection)

	print("Type 'exit' to leave the archives")

	while True:
		user_input = input("Pray enter your wish:").strip().lower()
        
		if user_input == "add ship":
			add_ship(connection)
		elif user_input == "add flag officer":
			make_flag_officer(connection)
		elif user_input == "add commissioned officer":
			make_commissioned_officer(connection)
		elif user_input == "add warrant officer":
			make_warrant_officer(connection)
		elif user_input == "add petty officer":
			make_petty_officer(connection)
		elif user_input == "add dispatch":
			add_dispatch(connection)
		elif user_input == "add enemy ship":
			add_enemy_ship(connection)
		elif user_input == "add squadron":
			add_squadron(connection)
		elif user_input == "add fleet":
			add_fleet(connection)
		elif user_input == "add report":
			add_report(connection)
		elif user_input == "add engagement":
			add_engagement(connection)
		elif user_input == "add station":
			add_station(connection)
		elif user_input == "add port":
			add_port(connection)
		elif user_input == "promote officer":
			promote_officer(connection)
		elif user_input == "exit":
			break
		else:
			print("Unknown command. Please try again.")

	print("Good luck, and may the wind fill your sails!")
	close_db(connection)

if __name__ == "__main__":
	main()

Greetings, welcome to the Naval Archives.


OperationalError: (1045, "Access denied for user 'root'@'localhost' (using password: YES)")